In [1]:
import requests
import zipfile
import pandas as pd 
import glob
from memory_profiler import memory_usage 

In [2]:
import os

In [3]:
%load_ext rpy2.ipython 
%load_ext memory_profiler 

D:\Anaconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


### Downloading the data

In [5]:
file_id = 26766812
x = requests.get(f'https://api.figshare.com/v2/file/download/{file_id}')

In [6]:
open('data.zip', 'wb').write(x.content)

814041183

In [7]:
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('data_extracted')

In [26]:
[file for file in os.listdir("data_extracted") if ".csv" in file]

['ACCESS-CM2_daily_rainfall_NSW.csv',
 'ACCESS-ESM1-5_daily_rainfall_NSW.csv',
 'AWI-ESM-1-1-LR_daily_rainfall_NSW.csv',
 'BCC-CSM2-MR_daily_rainfall_NSW.csv',
 'BCC-ESM1_daily_rainfall_NSW.csv',
 'CanESM5_daily_rainfall_NSW.csv',
 'CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 'CMCC-CM2-SR5_daily_rainfall_NSW.csv',
 'CMCC-ESM2_daily_rainfall_NSW.csv',
 'EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 'FGOALS-f3-L_daily_rainfall_NSW.csv',
 'FGOALS-g3_daily_rainfall_NSW.csv',
 'GFDL-CM4_daily_rainfall_NSW.csv',
 'GFDL-ESM4_daily_rainfall_NSW.csv',
 'INM-CM4-8_daily_rainfall_NSW.csv',
 'INM-CM5-0_daily_rainfall_NSW.csv',
 'KIOST-ESM_daily_rainfall_NSW.csv',
 'MIROC6_daily_rainfall_NSW.csv',
 'MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv',
 'MPI-ESM1-2-HR_daily_rainfall_NSW.csv',
 'MPI-ESM1-2-LR_daily_rainfall_NSW.csv',
 'MRI-ESM2-0_daily_rainfall_NSW.csv',
 'NESM3_daily_rainfall_NSW.csv',
 'NorESM2-LM_daily_rainfall_NSW.csv',
 'NorESM2-MM_daily_rainfall_NSW.csv',
 'observed_daily_rainfall_SYD.csv',
 'SAM0

In [27]:
csvfiles =  [file for file in os.listdir("data_extracted") if ".csv" in file]

In [28]:
csvfiles

['ACCESS-CM2_daily_rainfall_NSW.csv',
 'ACCESS-ESM1-5_daily_rainfall_NSW.csv',
 'AWI-ESM-1-1-LR_daily_rainfall_NSW.csv',
 'BCC-CSM2-MR_daily_rainfall_NSW.csv',
 'BCC-ESM1_daily_rainfall_NSW.csv',
 'CanESM5_daily_rainfall_NSW.csv',
 'CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 'CMCC-CM2-SR5_daily_rainfall_NSW.csv',
 'CMCC-ESM2_daily_rainfall_NSW.csv',
 'EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 'FGOALS-f3-L_daily_rainfall_NSW.csv',
 'FGOALS-g3_daily_rainfall_NSW.csv',
 'GFDL-CM4_daily_rainfall_NSW.csv',
 'GFDL-ESM4_daily_rainfall_NSW.csv',
 'INM-CM4-8_daily_rainfall_NSW.csv',
 'INM-CM5-0_daily_rainfall_NSW.csv',
 'KIOST-ESM_daily_rainfall_NSW.csv',
 'MIROC6_daily_rainfall_NSW.csv',
 'MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv',
 'MPI-ESM1-2-HR_daily_rainfall_NSW.csv',
 'MPI-ESM1-2-LR_daily_rainfall_NSW.csv',
 'MRI-ESM2-0_daily_rainfall_NSW.csv',
 'NESM3_daily_rainfall_NSW.csv',
 'NorESM2-LM_daily_rainfall_NSW.csv',
 'NorESM2-MM_daily_rainfall_NSW.csv',
 'observed_daily_rainfall_SYD.csv',
 'SAM0

### Combining data CSVs

In [30]:
%%time
%memit
big_df = []
for csvfile in csvfiles:
    df = pd.read_csv("./data_extracted/"+csvfile, index_col=None, header=0)
    df['model'] = csvfile.split("_")[0] 
    big_df.append(df)

frame = pd.concat(big_df, axis=0, ignore_index=True)

peak memory: 1061.02 MiB, increment: 0.00 MiB
Wall time: 1min 51s


In [31]:
frame

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.250000,-35.00000,140.625,142.500,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.250000,-35.00000,140.625,142.500,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.250000,-35.00000,140.625,142.500,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.250000,-35.00000,140.625,142.500,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.250000,-35.00000,140.625,142.500,1.047658e-02,ACCESS-CM2
...,...,...,...,...,...,...,...
62513858,2014-12-27 12:00:00,-30.157068,-29.21466,153.125,154.375,5.543748e-01,TaiESM1
62513859,2014-12-28 12:00:00,-30.157068,-29.21466,153.125,154.375,7.028577e+00,TaiESM1
62513860,2014-12-29 12:00:00,-30.157068,-29.21466,153.125,154.375,2.347570e-01,TaiESM1
62513861,2014-12-30 12:00:00,-30.157068,-29.21466,153.125,154.375,2.097459e+00,TaiESM1


In [32]:
frame.to_csv("combined_data.csv")
print("combined_data.csv file saved!")

combined_data.csv file saved!


#### Run time and memory Usage Discussion:

peak memory: 1061.02 MiB, increment: 0.00 MiB
Wall time: 1min 51s

peak memory: 151.94 MiB, increment: 0.08 MiB
CPU times: user 48 s, sys: 5.18 s, total: 53.2 s
Wall time: 55.6 s

peak memory: 935.61 MiB, increment: 0.11 MiB
CPU times: user 56.5 s, sys: 3.65 s, total: 1min
Wall time: 1min

peak memory: 1018.20 MiB, increment: 0.19 MiB
CPU times: user 1min 2s, sys: 12 s, total: 1min 14s
Wall time: 1min 20s

#### Discussion:

Running the combining data process is very RAM intensive. Our computers took around 1min to run and the peak memory usage was around 1000 MB.

### Load the combined CSV to memory and perform a simple EDA
1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts).
* Changing dtype of your data
* Load just columns what we want
* Loading in chunks
* Dask

2. Discuss your observations.

In [4]:
import dask.dataframe as dd

#### Loading in chunks for counting the model column

In [5]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
observed              46020
dtype: int32
peak memory: 3078.15 MiB, increment: 2934.44 MiB
Wall time: 1min 59s


#### Loading only the column we want for the EDA, which is the model column

In [7]:
%%time
%%memit
data = pd.read_csv("combined_data.csv",usecols=["model"])
print(data["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CMCC-CM2-HR4        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
GFDL-CM4            3219300
FGOALS-f3-L         3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
NESM3                966420
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
AWI-ESM-1-1-LR       966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
observed              46020
Name: model, dtype: int64
peak memory: 1424.96 MiB, increment: 953.43 MiB
Wall time: 1min 1s


#### Loading using Dask

In [8]:
%%time
%%memit

ddf = dd.read_csv('combined_data.csv')
print(ddf["model"].value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
NorESM2-MM          3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
observed              46020
Name: model, dtype: int64
peak memory: 1943.04 MiB, increment: 995.03 MiB
Wall time: 1min 2s


### Discussion

In this section, we experimented with loading using chunk size of 10,000,000, loading using only the column we wanted, and loading using Dask.  

Comparing the results:
* Loading using chunk size of 10,000,000:  
peak memory: 3078.15 MiB, increment: 2934.44 MiB
Wall time: 1min 59s

* Loading using only the column we wanted:  
peak memory: 1424.96 MiB, increment: 953.43 MiB
Wall time: 1min 1s

* Loading using Dask:  
peak memory: 1943.04 MiB, increment: 995.03 MiB
Wall time: 1min 2s

From the above runtime stats, we can observe that loading using chunk size takes the most of the memory and longer wall time comparing to loading using only the column we wanted and loading using Dask. Loading using only the column we wanted has the minimal number of peak memory usage and wall time comparing to loading using Dask.


### Perform a simple EDA in R

Pick an approach to transfer the dataframe from python to R.
* Parquet file
* Feather file (We chose this one)
* Pandas exchange
* Arrow exchange

### Discuss why you chose this approach over others.

From the lecture notes, feather is faster than the parquet or arrow format in writing to files. We picked it because it's convenient to use and it's very common to use feather with R programming language.

In [4]:
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra
import pyarrow.feather as feather

In [5]:
%%R
#just seeing if its available
library("arrow")
library("dplyr")

R[write to console]: Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar

R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [6]:
%%time
%%memit

dataset = ds.dataset("combined_data.csv", format="csv")
table = dataset.to_table()

peak memory: 4653.75 MiB, increment: 4378.27 MiB
Wall time: 1min 42s


In [7]:
%%time

# experiment in writing in feather format 
feather.write_feather(table, 'combined_data.feather')

Wall time: 10.9 s


In [6]:
%%time
%%R

### here we are showing how much time it took to read a feather file what we wrote in python
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("combined_data.feather", col_select=c("model"))
print(class(r_table))
library(dplyr)
result <- count(r_table, model)
end_time <- Sys.time()
print(end_time - start_time)
print(result)

[1] "tbl_df"     "tbl"        "data.frame"
Time difference of 4.992996 secs
# A tibble: 28 x 2
   model                  n
 * <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 18 more rows
Wall time: 5.47 s
